In [ ]:

import numpy as np # linear algebra
import pandas as pd# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
  # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
%matplotlib inline

Insurance = pd.read_csv(r'../input/porto-seguro-safe-driver-prediction/train.csv') 
test1=pd.read_csv(r'../input/porto-seguro-safe-driver-prediction/test.csv')
#print (Insurance.shape)
print(test1.shape)
#Insurance.head()
test1.head()


In [ ]:
# Graph to show the Higly Imbalanced Data. To Predict the Column ['target']. Right now.. majority of records/rows with 'Target=0'
# And few records with 'Target = 1'
pd.value_counts(Insurance['target']).plot.bar()
plt.title('Target class histogram')
plt.xlabel('target')


In [ ]:
# Top 5 features with most missing data in training dataset
Insurance = Insurance.replace(-1, np.nan)
Insurance_missing = Insurance.isnull().sum().sort_values(ascending=False)
Insurance_missing = pd.DataFrame(Insurance_missing).reset_index()
Insurance_missing.columns = ['Feature', 'Number of Data Records Missing']
Insurance_missing.head()


In [ ]:
# Top 5 features with most missing data in test dataset
test1 = test1.replace(-1, np.nan)
test1_missing = test1.isnull().sum().sort_values(ascending=False)
test1_missing = pd.DataFrame(test1_missing).reset_index()
test1_missing.columns = ['Feature', 'Number of Data Records Missing']
test1_missing.head()

In [ ]:
# drop features 'ps_car_03_cat' and 'ps_car_05_cat'
Insurance = Insurance.drop(['ps_car_03_cat', 'ps_car_05_cat'], axis = 1)

Insurance.dtypes

In [ ]:
# drop features 'ps_car_03_cat' and 'ps_car_05_cat'
test1 = test1.drop(['ps_car_03_cat', 'ps_car_05_cat'], axis = 1)

test1.dtypes

In [ ]:
# visualize the correlation between remaining features
import seaborn as sns
corr = Insurance.corr()
sns.heatmap(corr)
plt.title("Correlation Between Features", fontsize = 14)
plt.xticks(rotation = 'vertical')
plt.figure(figsize = (20,10), dpi= 110)
plt.show()

In [ ]:
# drop features with '_calc_' in feature names, because they have low correlation
feature_calc = list(Insurance.columns[Insurance.columns.str.contains('_calc_')])
Insurance = Insurance.drop(feature_calc, axis = 1)
print (Insurance.shape)


In [ ]:
# drop features with '_calc_' in feature names, because they have low correlation
feature_calc = list(test1.columns[test1.columns.str.contains('_calc_')])
test1 = test1.drop(feature_calc, axis = 1)
print (test1.shape)

In [ ]:
# replace missing data in the remaining features
feature_bin = [f for f in Insurance.columns if f.endswith('bin')] 
feature_cat = [f for f in Insurance.columns if f.endswith('cat')] 
feature_els = [f for f in Insurance.columns if (f not in feature_bin) & (f not in feature_cat) & (f not in ['id', 'target'])]


In [ ]:
# replace missing data in the remaining features_test
feature_bin_test1= [f for f in test1.columns if f.endswith('bin')] 
feature_cat_test1 = [f for f in test1.columns if f.endswith('cat')] 
feature_els_test1 = [f for f in test1.columns if (f not in feature_bin_test1) & (f not in feature_cat_test1) & (f not in ['id'])]


In [ ]:
# filling missing data with mode value for binary and categorical Data.

for f in (feature_bin + feature_cat):
    Insurance[f].fillna(value=Insurance[f].mode()[0], inplace=True)
   
# filling missing data with mean value.
for f in feature_els:
    Insurance[f].fillna(value=Insurance[f].mean(), inplace=True)


In [ ]:
# filling missing data with mode value for binary and categorical Data_test.

for f in (feature_bin_test1 + feature_cat_test1):
    test1[f].fillna(value=test1[f].mode()[0], inplace=True)
   
# filling missing data with mean value.
for f in feature_els:
    test1[f].fillna(value=test1[f].mean(), inplace=True)


In [ ]:
# Visualize binary features data and categorical features data (for styling, plot only features with 2 unique values)
plt.figure(figsize=(20, 20), dpi = 110)

count = 0

for f in (feature_bin + feature_cat):
    if  Insurance[f].nunique() <= 2:
        ax = plt.subplot(4,4,count+1)
        sns.countplot(Insurance[f])
        plt.xlabel(f, fontsize = 14)
        plt.ylabel("Count", fontsize = 12)
       # plt.ylim(0, 600000)
        count += 1
        
plt.show()

# Feature Data Distribution Subplots

In [ ]:
# Class count
count_class_0, count_class_1 = Insurance.target.value_counts()

# Divide by class
df_class_0 = Insurance[Insurance['target'] == 0]
df_class_1 = Insurance[Insurance['target'] == 1]
print('Zero',count_class_0)
print('One',count_class_1)

In [ ]:
# Doing Random Undersampling 
df_class_0_under = df_class_0.sample(count_class_1)
df_test_under = pd.concat([df_class_0_under, df_class_1], axis=0)

print('After Random under-sampling:')
print(df_test_under.target.value_counts())

df_test_under.target.value_counts().plot(kind='bar', title='Count (target)');

In [ ]:
df_test_under.dtypes

In [ ]:
df_test_under.shape

In [ ]:
# From Now modelling with Undersampled and balanced Data
train2=df_test_under
y_train = train2['target']
X_train = train2.drop(['target','id'], axis=1)
print(X_train.shape)
#print(X.dtypes)
print(y_train.shape)
#print(X_train)

In [ ]:

X_test1 = test1.drop(['id'], axis=1)
print(X_test1.shape)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test1 = sc.transform(X_test1)


In [ ]:
### Model Number Two Random Forest #####
from sklearn.ensemble import RandomForestClassifier
rclf = RandomForestClassifier()
rclf.fit(X_train,y_train)
y_pred = rclf.predict(X_test1)

In [ ]:
sample=pd.read_csv(r'../input/porto-seguro-safe-driver-prediction/sample_submission.csv')
sample['target']=y_pred
sample.to_csv("sample_submission.csv",index=False)
print(sample.head(20))